In [1]:
import numpy as np
import bchlib
import binascii
import math
from pydtmc import MarkovChain
from scipy import sparse
from scipy.stats import rv_discrete
import sys
np.set_printoptions(threshold=sys.maxsize)
from scipy.stats import bernoulli as bern
import BCH_BSC_Functions as func

In [2]:
S=32
M=32
n=20
BCH_Poly=37
Bits_per_error=5
p=0.05
M=32

size=20000
samples=10000

codes_combined=func.code_combined(S,M,n,BCH_Poly,Bits_per_error)
codes_punctured=func.code_punctured(S,M,n,BCH_Poly,Bits_per_error)

np.random.seed(0)
trans_mat1=sparse.rand(S,S,density=0.25)
trans_mat1=trans_mat1.todense()
trans_mat1=trans_mat1/np.sum(trans_mat1,axis=1)

np.random.seed(1)
trans_mat2=sparse.rand(S,S,density=0.25)
trans_mat2=trans_mat2.todense()
trans_mat2=trans_mat2/np.sum(trans_mat2,axis=1)

In [3]:
states_numbers=np.arange(S)
states=np.empty(size,dtype=int)
messages=np.empty(size,dtype=int)
np.random.seed(0)
states[0]=np.random.randint(M)
for i in range(1,size):
    prob=np.array((trans_mat1*(1-i/size)+trans_mat2*(i/size))[states[i-1],:]).squeeze()
    custm=rv_discrete(values=(states_numbers,prob))
    states[i]=custm.rvs(size=1)
    
for i in range(0,size):
    messages[i]=np.random.randint(M)

In [4]:
sent_punctured=func.encoder(M,states,messages,codes_punctured,size)
#sent_combined=func.encoder(M,states,messages,codes_combined,size)

In [5]:
np.random.seed(0)
received_punctured=func.channel(sent_punctured,size,p)
#np.random.seed(0)
#received_combined=func.channel(sent_combined,size,p)

In [6]:
transitions_init=0.1

In [7]:
#codes=codes_combined
codes=codes_punctured

#received=received_combined
received=received_punctured

In [8]:
state_predicted=np.empty(size,dtype=int)
message_predicted=np.empty(size,dtype=int)

transitions=np.full((S,S),transitions_init)
trans_mat_predicted=transitions/np.sum(transitions,axis=1)
startprob=np.full(S,1/S)

B=np.empty(S)
message_number=np.empty((S,size),dtype=int)
T1=np.empty(S)
prev_T1=np.empty(S)


#First packet
packet=received[0]
compare=(codes!=packet)
hammdist=np.sum(compare,axis=1)
for l in range(0,S):
    index=np.argmin(hammdist[M*l:M*l+M])
    message_number[l,0]=index
    B[l]=np.sum(np.power(p,hammdist[M*l:M*l+M])*np.power((1-p),n-hammdist[M*l:M*l+M]))
T1=startprob*B
    
state_predicted[0]=np.argmax(T1)
message_predicted[0]=message_number[state_predicted[0],0]


for j in range(1,size):
    prev_T1=np.array(T1)
    packet=received[j]
    compare=(codes!=packet)
    hammdist=np.sum(compare,axis=1)
    for l in range(0,S):
        index=np.argmin(hammdist[M*l:M*l+M])
        message_number[l,j]=index
        B[l]=np.sum(np.power(p,hammdist[M*l:M*l+M])*np.power((1-p),n-hammdist[M*l:M*l+M]))
        apriori=np.sum(prev_T1*trans_mat_predicted[:,l])
        #apriori=np.sum(prev_T1*transitions[:,l])
        T1[l]=apriori*B[l]
    if(T1[0]<10**(-100)):
        T1=T1*10**100

    state_predicted[j]=np.argmax(T1)
    message_predicted[j]=message_number[state_predicted[j],j]    
    
    transitions[state_predicted[j-1],state_predicted[j]]+=1
    trans_mat_predicted=transitions/np.sum(transitions,axis=1)


In [9]:
print(func.errors(states,messages,state_predicted,message_predicted,size))
print(func.errors(states[10000:20000],messages[10000:20000],state_predicted[10000:20000],message_predicted[10000:20000],10000))
print(func.errors(states[0:10000],messages[0:10000],state_predicted[0:10000],message_predicted[0:10000],10000))

(508, 520)
(245, 249)
(263, 271)


In [10]:
delay=1

state_predicted=np.empty(size,dtype=int)
message_predicted=np.empty(size,dtype=int)

transitions=np.full((S,S),transitions_init)
trans_mat_predicted=transitions/np.sum(transitions,axis=1)
startprob=np.full(S,1/S)

startprob=np.full(S,1/S)
message_number=np.empty((S,size),dtype=int)
forward=np.empty((S,size))
backward=np.zeros((S,delay+1))
B=np.empty((S,size))
alpha=np.empty(S)

packet=received[0]
compare=(codes!=packet)
hammdist=np.sum(compare,axis=1)
for l in range(0,S):
    index=np.argmin(hammdist[M*l:M*l+M])
    message_number[l,0]=index
    B[l,0]=np.sum(np.power(p,hammdist[M*l:M*l+M])*np.power((1-p),n-hammdist[M*l:M*l+M]))

forward[:,0]=startprob*B[:,0]


for j in range(1,size):
    backward.fill(0)
    packet=received[j]
    compare=(codes!=packet)
    hammdist=np.sum(compare,axis=1)
    for l in range(0,S):
        index=np.argmin(hammdist[S*l:S*l+S])
        message_number[l,j]=index
        B[l,j]=np.sum(np.power(p,hammdist[S*l:S*l+S])*np.power((1-p),n-hammdist[S*l:S*l+S]))

    for i in range(0,S):
        apriori=np.sum(forward[:,j-1]*trans_mat_predicted[:,i])
        forward[i,j]=apriori*B[i,j]

    if(forward[0,j]<10**(-100)):
        forward[:,j]=forward[:,j]*10**100

    backward[:,delay].fill(1)
    for k in range(delay-1,-1,-1):
        for i in range(0,S):
            alpha=backward[:,k+1]*trans_mat_predicted[i,:]
            backward[i,k]=np.sum(alpha*B[:,j+k-delay+1])

    state_predicted[j-delay]=np.argmax(forward[:,j-delay]*backward[:,0])
    if(j-delay>=1):
        transitions[state_predicted[j-delay-1],state_predicted[j-delay]]+=1
    trans_mat_predicted=transitions/np.sum(transitions,axis=1)
    

    
for i in range(size-delay,size):
    state_predicted[i]=np.argmax(forward[:,i]*backward[:,delay+i-size])

for i in range(0,size):
    message_predicted[i]=message_number[state_predicted[i],i]

In [11]:
print(func.errors(states,messages,state_predicted,message_predicted,size))
print(func.errors(states[10000:20000],messages[10000:20000],state_predicted[10000:20000],message_predicted[10000:20000],10000))
print(func.errors(states[0:10000],messages[0:10000],state_predicted[0:10000],message_predicted[0:10000],10000))

(275, 296)
(126, 136)
(149, 160)


In [10]:
#Exactly know trans_mat at all times

state_predicted=np.empty(size,dtype=int)
message_predicted=np.empty(size,dtype=int)

transitions=np.full((S,S),transitions_init)
trans_mat_predicted=transitions/np.sum(transitions,axis=1)
startprob=np.full(S,1/S)

B=np.empty(S)
message_number=np.empty((S,size),dtype=int)
T1=np.empty(S)
prev_T1=np.empty(S)


#First packet
packet=received[0]
compare=(codes!=packet)
hammdist=np.sum(compare,axis=1)
for l in range(0,S):
    index=np.argmin(hammdist[M*l:M*l+M])
    message_number[l,0]=index
    B[l]=np.sum(np.power(p,hammdist[M*l:M*l+M])*np.power((1-p),n-hammdist[M*l:M*l+M]))
T1=startprob*B
    
state_predicted[0]=np.argmax(T1)
message_predicted[0]=message_number[state_predicted[0],0]


for j in range(1,size):
    trans_mat_predicted=(trans_mat1*(1-j/size)+trans_mat2*(j/size))
    prev_T1=np.array(T1)
    packet=received[j]
    compare=(codes!=packet)
    hammdist=np.sum(compare,axis=1)
    for l in range(0,S):
        index=np.argmin(hammdist[M*l:M*l+M])
        message_number[l,j]=index
        B[l]=np.sum(np.power(p,hammdist[M*l:M*l+M])*np.power((1-p),n-hammdist[M*l:M*l+M]))
        apriori=np.sum(prev_T1*trans_mat_predicted[:,l])
        #apriori=np.sum(prev_T1*transitions[:,l])
        T1[l]=apriori*B[l]
    if(T1[0]<10**(-100)):
        T1=T1*10**100

    state_predicted[j]=np.argmax(T1)
    message_predicted[j]=message_number[state_predicted[j],j]    
    


In [11]:
print(func.errors(states,messages,state_predicted,message_predicted,size))
print(func.errors(states[10000:20000],messages[10000:20000],state_predicted[10000:20000],message_predicted[10000:20000],10000))
print(func.errors(states[0:10000],messages[0:10000],state_predicted[0:10000],message_predicted[0:10000],10000))

(425, 438)
(226, 231)
(199, 207)


In [14]:
#Exactly know trans_mat at all times

delay=1

state_predicted=np.empty(size,dtype=int)
message_predicted=np.empty(size,dtype=int)

transitions=np.full((S,S),transitions_init)
trans_mat_predicted=transitions/np.sum(transitions,axis=1)
startprob=np.full(S,1/S)

startprob=np.full(S,1/S)
message_number=np.empty((S,size),dtype=int)
forward=np.empty((S,size))
backward=np.zeros((S,delay+1))
B=np.empty((S,size))
alpha=np.empty(S)

packet=received[0]
compare=(codes!=packet)
hammdist=np.sum(compare,axis=1)
for l in range(0,S):
    index=np.argmin(hammdist[M*l:M*l+M])
    message_number[l,0]=index
    B[l,0]=np.sum(np.power(p,hammdist[M*l:M*l+M])*np.power((1-p),n-hammdist[M*l:M*l+M]))

forward[:,0]=startprob*B[:,0]


for j in range(1,size):
    trans_mat_predicted=trans_mat1*(1-j/size)+trans_mat2*(j/size)
    backward.fill(0)
    packet=received[j]
    compare=(codes!=packet)
    hammdist=np.sum(compare,axis=1)
    for l in range(0,S):
        index=np.argmin(hammdist[S*l:S*l+S])
        message_number[l,j]=index
        B[l,j]=np.sum(np.power(p,hammdist[S*l:S*l+S])*np.power((1-p),n-hammdist[S*l:S*l+S]))

    for i in range(0,S):
        apriori=np.sum(forward[:,j-1]*trans_mat_predicted[:,i])
        forward[i,j]=apriori*B[i,j]

    if(forward[0,j]<10**(-100)):
        forward[:,j]=forward[:,j]*10**100

    backward[:,delay].fill(1)
    for k in range(delay-1,-1,-1):
        for i in range(0,S):
            beta=np.array(trans_mat_predicted[i,:]).squeeze()
            alpha=backward[:,k+1]*beta
            backward[i,k]=np.sum(alpha*B[:,j+k-delay+1])

    state_predicted[j-delay]=np.argmax(forward[:,j-delay]*backward[:,0])    

    
for i in range(size-delay,size):
    state_predicted[i]=np.argmax(forward[:,i]*backward[:,delay+i-size])

for i in range(0,size):
    message_predicted[i]=message_number[state_predicted[i],i]

In [15]:
print(func.errors(states,messages,state_predicted,message_predicted,size))
print(func.errors(states[10000:20000],messages[10000:20000],state_predicted[10000:20000],message_predicted[10000:20000],10000))
print(func.errors(states[0:10000],messages[0:10000],state_predicted[0:10000],message_predicted[0:10000],10000))

(190, 215)
(92, 103)
(98, 112)
